In [1]:
from selenium import webdriver
import time
driver = webdriver.Chrome('C:\chromedriver.exe')

In [2]:
driver.get('https://www.melon.com/')

In [3]:
# melon Dj 카테고리 선택
melon_dj = driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[4]/a/span[2]').click()

In [4]:
# 태그 검색
song_info = {}
weather = '비오는날'# 날씨 조건문 설정

driver.implicitly_wait(100)
tag_input = driver.find_element_by_class_name('input_text')
tag_input.send_keys(weather)
xpath = driver.find_element_by_xpath('//*[@id="conts"]/div[1]/div[2]/div[2]/button')
driver.execute_script("arguments[0].click();", xpath)                                     # selenium에서 클릭이 안 될때 클릭 하는 법


In [5]:
# 개별 palylist에서 노래 정보 크롤링 함수
def info_select():
    body = driver.find_element_by_tag_name('tbody')
    trs = body.find_elements_by_tag_name('tr')   #노래 정보 tag


    titles = []
    singer = []
    likes = []


    for tr in trs:
        # 노래제목, 가수 각각 리스트에 저장
        name = tr.find_element_by_css_selector('div.ellipsis.rank01 a').text
        artist = tr.find_element_by_css_selector('div.ellipsis.rank02 a').text
        titles.append(name)
        singer.append(artist)

        # 좋아요 수 리스트에 저장
        cnt = tr.find_element_by_css_selector('span.cnt').text
        cnt = int(float(cnt.replace(',', '')))                   #문자열 숫자로 변환
        likes.append(cnt)
        #break

    # (노래제목, 가수) 튜플 생성
    info = list(zip(titles,singer))
    #print(info)

    # {(노래제목, 가수) : 좋아요 수 } 딕셔너리 생성
    for i in range(len(likes)):
        if info[i] in song_info:   # 같은 노래와 가수가 있으면 50점 추가
            song_info[info[i]] += 50
        else:
            song_info[info[i]] = likes[i]


In [6]:
# selenium time 생성
driver.implicitly_wait(100)

In [7]:
#플레이 리스트 목록
dj_play_list = driver.find_element_by_xpath('//*[@id="djPlylstList"]/div')
lis = dj_play_list.find_elements_by_tag_name('li')
lis_len = len(lis)

# 각 플레이 리스트 들어갔다 나오기
for i in range(lis_len):
    dj_play_list = driver.find_element_by_xpath('//*[@id="djPlylstList"]/div')  # 한 플레이 리스트를 들어갔다 나오면 정보가 새로 바뀌므로 오류 발생, for문 안에 lis 생성하여 저장
    lis = dj_play_list.find_elements_by_tag_name('li')
    elem = lis[i].find_element_by_css_selector('a')
    driver.implicitly_wait(100)
    driver.execute_script("arguments[0].click();", elem)  # play list click
    info_select()    # (노래제목,가수),좋아요 함수 호출
    driver.back()    # 플레이 리스트에서 목록 페이지로 돌아가기


In [8]:
# 좋아요 수 기준으로 내림차순 만들기
song_info = sorted(song_info.items(), key=lambda x: x[1], reverse=True)
#print(song_info)

[(('모든 날, 모든 순간 (Every day, Every Moment)', '폴킴'), 332083), (('너를 만나', '폴킴'), 260448), (('비', '폴킴'), 251429), (('비도 오고 그래서 (Feat. 신용재)', '헤이즈 (Heize)'), 248213), (('흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야', '장범준'), 243448), (('bad guy', 'Billie Eilish'), 227500), (('바람기억', '나얼'), 214974), (('Paris In The Rain', 'Lauv'), 208681), (('그대라는 시', '태연 (TAEYEON)'), 169029), (('미안해', '양다일'), 169029), (('안녕', '폴킴'), 166212), (('걱정말아요 그대', '이적'), 164359), (('소나기', '아이오아이 (I.O.I)'), 163845), (('나의 옛날이야기', '아이유'), 158558), (('Rain', '태연 (TAEYEON)'), 158092), (('180도', '벤'), 155509), (('마음을 드려요', '아이유'), 150493), (('늦은 밤 너의 집 앞 골목길에서', '노을'), 145317), (('술이 문제야', '장혜진'), 143158), (('Señorita', 'Shawn Mendes'), 142540), (('그건 아마 우리의 잘못은 아닐 거야', '백예린'), 137546), (('좋아', '민서'), 135499), (('비가와 (Rain)', '소유 (SOYOU)'), 129018), (('부담이 돼 (Feat. 휘인 Of 마마무)', '정키'), 125605), (('신용재', '하은'), 125433), (('솔직하게 말해서 나', '김나영'), 122834), (('idontwannabeyouanymore', 'Billie Eilish'), 116149), (('너도 그냥 날 놓아주면 돼', '윤건'), 115825), 

In [10]:
#결과 출력
result = []
for i in range(5):
    rank = i+1
    title_result = song_info[i][0][0]
    singer_result = song_info[i][0][1]
    likes_result = song_info[i][1]
    #print(str(rank)+'위.',title_result,singer_result,likes_result)
    
    result.append([rank,title_result,singer_result,likes_result])
result

[[1, '모든 날, 모든 순간 (Every day, Every Moment)', '폴킴', 332083],
 [2, '너를 만나', '폴킴', 260448],
 [3, '비', '폴킴', 251429],
 [4, '비도 오고 그래서 (Feat. 신용재)', '헤이즈 (Heize)', 248213],
 [5, '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야', '장범준', 243448]]